In [16]:
# original code
import torch
from tqdm import tqdm
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv,GATConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import VGAE

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

def decode(model,epochs,out_channels,data,device):
    num_features = data.num_node_features
    x = data.x.to(device)
    edge_index = data.edge_index.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model.train()
    loss = 0
    for epoch in tqdm(range(1, epochs + 1)): # train
        all_loss = []
        optimizer.zero_grad()
        z = model.encode(x, edge_index)
        recon_loss = model.recon_loss(z, edge_index) # how to calculate the loss
        loss = recon_loss +20* (1 / data.num_nodes) * model.kl_loss()  # new line
        #print("all loss is ",float(loss))
        all_loss.append(loss)
        loss.backward()
        if(epoch%10==0):
            print("epoch is {} and loss is {}".format(epoch,loss))
        optimizer.step()
    model.eval()
    with torch.no_grad(): # pred
        z= model.encode(x, edge_index)
        print("z",z)
        print("z.shape",z.shape) # torch.Size([42372, 128])
    return z

class VGAE_gat(torch.nn.Module):
    def __init__(self, in_channels, out_channels,heads):
        super(VGAE_gat, self).__init__()      
        self.conv1 = GATConv(in_channels, out_channels,heads[0]) # cached only for transductive learning
        self.conv2 = GATConv(heads[0]*out_channels, out_channels,heads[1]) # cached only for transductive learning
        self.conv_mu = GATConv(heads[1]* out_channels, out_channels,heads[2],concat = False)
        self.conv_logstd = GATConv(heads[1]* out_channels, out_channels, heads[2],concat =False)
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

# 数据为边和每条边的特征
with open("./transitions.txt") as f:
    edges1 = [list(map(int,[line.split()[0],line.split()[1]])) for line in f.readlines()]
    # edges2 = [list(map(int,[line.split()[1],line.split()[0]])) for line in f.readlines()]
    # edges = edges1+edges2
    edges = edges1
    # print(len(edges))
    # print((edges1))

edges = torch.tensor(edges,dtype=torch.long)
print(edges.shape)
edges = edges.transpose(0,1)
print(edges.shape) 

from torch_geometric.data import Data
import numpy as np

lac_features = np.load('./lac_feature.npy')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

out_dims = 64
data = Data(x= torch.tensor(lac_features,dtype=torch.float),edge_index=edges) # 修改其中的features

num_features = data.num_node_features

model = VGAE(VGAE_gat(num_features, out_dims,[2,2,1]))  # new line 修改其中的num_features
model = model.to(device)
z_gat= decode(model,100,out_dims,data,device) # 修改其中的data

torch.Size([603136, 2])
torch.Size([2, 603136])


 10%|█         | 10/100 [00:07<01:08,  1.31it/s]

epoch is 10 and loss is 1.9685161113739014


 20%|██        | 20/100 [00:15<01:00,  1.31it/s]

epoch is 20 and loss is 1.59755539894104


...

epoch is 80 and loss is 1.404968023300171


 90%|█████████ | 90/100 [01:07<00:07,  1.39it/s]

epoch is 90 and loss is 1.4001895189285278


100%|██████████| 100/100 [01:14<00:00,  1.34it/s]

epoch is 100 and loss is 1.394952416419983
z tensor([[ 0.4577, -0.0781, -0.1967,  ..., -0.3044, -0.1000, -0.1807],
        [ 0.4048, -0.0862, -0.1827,  ..., -0.3044, -0.1019, -0.1752],
        [ 0.5109, -0.0930, -0.2263,  ..., -0.3563, -0.1187, -0.2124],
        ...,
        [ 0.0610, -0.0053,  0.0918,  ...,  0.0190, -0.0214,  0.1146],
        [-0.0524, -0.0390,  0.1088,  ..., -0.0215,  0.0265,  0.0842],
        [ 0.0166,  0.0491, -0.0171,  ..., -0.0191,  0.0028,  0.0273]],
       device='cuda:0')
z.shape torch.Size([6053, 64])


In [3]:
z_gat.shape

torch.Size([6053, 64])

In [13]:
# 打开一个文本文件以写入模式
with open('output.txt', 'w') as file:
    first_line = f"{z_gat.shape[0]} {z_gat.shape[1]}\n"
    file.write(first_line)

    # 遍历每一行，对每行元素求和后写入
    for index, row in enumerate(z_gat):
        row_value = " ".join(str(value.item()) for value in row)
        line = f"{index} {row_value}\n"
        file.write(line)